## TripleTen Code Jam 2023
**Team: Santa's Coders**
- Marco Fernstaedt
- Veronica Steele
- Andrew Kwon

In this project, our team designed a delivery route application where users can select a start location and up to seven destinations. This project applies various methods to solving/estimating a small scale adaptation of the traveling salesman problem, or shortest Hamiltonian cycle.

### Libraries, Packages, Modules

The required libraries are contained in the route.py module, and are as follows:
- pandas (2.1.2)
- import random
- import math
- itertools.permutations
- geopy.distance (2.4.1)

In [1]:
import time
from route import Locations, Route

### Initializing Data

We'll create a Locations object from the dataset. We'll pick an arbitrary start city (Phoenix) and randomly select seven destination cities.

In [2]:
filepath = 'datasets\cities_final.csv'
cities = Locations(filepath)

home = 'Phoenix'
dest_cities = cities.get_all_cities().sample(7, random_state=42)['City'].to_list()
print(f'Home: {home}')
print(f'Cities: {dest_cities}')

Home: Phoenix
Cities: ['Memphis', 'Raleigh', 'Detroit', 'Miami', 'Portland', 'Atlanta', 'Fort Worth']


Given the start city and destination nodes, we can generate graph edges, edge weights, and coordinates for each city location. Additionally, we can initialize a Route object to begin calculating various routes and their associated costs.

In [3]:
edges, weights, coords = cities.create_map(home, dest_cities)
route = Route(home, dest_cities, edges, weights)

### Route Algorithms

To illustrate optimization improvements, we'll compare the following search path algorithms:  
- Random path (baseline)
- Greedy
- K-optimal Tours (Heuristic)
- Brute Force

**Random Route**

In [4]:
t0 = time.time()
path, cost = route.get_random_rte()
t1 = time.time()

print(f'Random Route: {path}')
print(f'Total Distance: {"{:.1f}".format(cost)} nm')
print(f'Calculation Time: {t1 - t0} seconds')

Random Route: ['Phoenix', 'Raleigh', 'Memphis', 'Fort Worth', 'Detroit', 'Portland', 'Miami', 'Atlanta', 'Phoenix']
Total Distance: 9466.4 nm
Calculation Time: 0.0 seconds


**Greedy Route**

In [5]:
t0 = time.time()
path, cost = route.get_greedy_rte()
t1 = time.time()

print(f'Greedy Route: {path}')
print(f'Total Distance: {"{:.1f}".format(cost)} nm')
print(f'Calculation Time: {t1 - t0} seconds')

Greedy Route: ['Phoenix', 'Fort Worth', 'Memphis', 'Atlanta', 'Raleigh', 'Detroit', 'Miami', 'Portland', 'Phoenix']
Total Distance: 6408.1 nm
Calculation Time: 0.0009980201721191406 seconds


**Shortest Route**

In [6]:
t0 = time.time()
path, cost = route.get_exact_rte()
t1 = time.time()

print(f'Shortest Route: {path}')
print(f'Total Distance: {"{:.1f}".format(cost)} nm')
print(f'Calculation Time: {t1 - t0} seconds')

Shortest Route: ['Phoenix', 'Fort Worth', 'Memphis', 'Atlanta', 'Miami', 'Raleigh', 'Detroit', 'Portland', 'Phoenix']
Total Distance: 5585.2 nm
Calculation Time: 0.015957355499267578 seconds


**Heuristic Route**

Comparison between baseline randomized route and optimization method

In [7]:
t0 = time.time()
rand_rte, cost = route.get_random_rte()
t1 = time.time()

print(f'Random Route: {rand_rte}')
print(f'Total Distance: {"{:.1f}".format(cost)} nm')
print(f'Calculation Time: {t1 - t0} seconds')

t0 = time.time()
opt_rte, new_cost = route.get_2_opt_rte(rand_rte)
t1 = time.time()

print(f'Heuristic Route: {opt_rte}')
print(f'Total Distance: {"{:.1f}".format(new_cost)} nm')
print(f'Calculation Time: {t1 - t0} seconds')

Random Route: ['Phoenix', 'Portland', 'Detroit', 'Atlanta', 'Miami', 'Fort Worth', 'Raleigh', 'Memphis', 'Phoenix']
Total Distance: 7221.6 nm
Calculation Time: 0.0 seconds
Heuristic Route: ['Phoenix', 'Portland', 'Detroit', 'Raleigh', 'Miami', 'Atlanta', 'Memphis', 'Fort Worth', 'Phoenix']
Total Distance: 5585.2 nm
Calculation Time: 0.0009965896606445312 seconds
